<a href="https://colab.research.google.com/github/AnnaZhuravleva/compling/blob/master/assignment_1/assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1

Using text http://www.gutenberg.org/files/2600/2600-0.txt
1. Make text lowercase and remove all punctuation except spaces and dots.
2. Tokenize text by BPE with vocab_size = 100
3. Train 3-gram language model with laplace smoothing $\delta=1$
4. Using beam search with k=10 generate sequences of length=10 conditioned on provided inputs. Treat dots as terminal tokens.
5. Calculate perplexity of the language model for the first sentence.

In [0]:
import string
import re
import collections
from collections import Counter
import nltk
from sklearn.base import TransformerMixin
import numpy as np
import math

In [92]:

from google.colab import drive
drive.mount('/content/drive')
text = open('/content/drive/My Drive/Colab Notebooks/compling/assignment_1/peace.txt')
# text = open('peace.txt', 'r', encoding='utf-8-sig')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [93]:
text = text.read()
len(text)

3227581

In [94]:
def preprocess_text(text):
    punct = list(set(re.findall('\W', text)))
    punct.remove('.')
    words = ''
    for word in text.split():
        for letter in word:
            if letter not in punct:
                words += letter
        words += ' '
    text = words.lower()
        
    # replace all punctuation except dots with spaces
    # collapse multiple spaces into one '   ' -> ' '
    return text


text = preprocess_text(text)
len(text)
# assert len(text) == 3141169

3130570

In [0]:
text = text.split('.')
text = [x.strip() for x in text]
text = [t + '.' for t in text]

In [0]:
class BPE(TransformerMixin):
    
    def __init__(self, vocab_size=100):
        super(BPE, self).__init__()
        self.vocab_size = vocab_size
        # index to token
        self.itos = []
        # token to index
        self.stoi = {}
        
    def fit(self, text):
        """
        fit itos and stoi
        text: list of strings 
        """
        
        # TODO
        # tokenize text by symbols and fill in self.itos and self.stoi
        self.itos = ['.']
        for item in set([item for txt in text for item in set(txt) if item != '.']):
          self.itos.append(item)
        self.stoi = {l: idx for idx, l in enumerate(self.itos)}
        text = [[self.stoi[letter] for letter in txt] for txt in text]
        
        while len(self.itos) < self.vocab_size:
            # TODO
            # count bigram freqencies in the text
            bigrams = collections.Counter()
            for txt in text:
                i = 0
                while i + 1 < len(txt):
                    bigrams[(txt[i], txt[i+1])] += 1
                    i += 1
                
                
            mc = bigrams.most_common(1)[0][0]
            new_token = str(self.itos[int(mc[0])]) + str(self.itos[int(mc[1])])
            new_id = len(self.itos)
            
            self.itos.append(new_token)
            self.stoi[new_token] = new_id
            
            # find occurences of the new_token in the text and replace them with new_id
            tmp = []
            for txt in text:
                tmp2 = []
                i = 0
                while i + 1 < len(txt):
                    if new_token == self.itos[txt[i]] + self.itos[txt[i+1]]:
                        tmp2.append(new_id)
                        i += 2
                    else:
                        tmp2.append(txt[i])
                        i += 1
                tmp.append(tmp2)
                    
            text = tmp 
           
        return self
    
    def transform(self, text):
        """
        convert text to a sequence of token ids
        text: list of strings
        """ 
        max_size = max([len(tok) for tok in self.itos])
        
        new_text = []
        for txt in text:
            i = 0
            new_txt = []
            while i < len(txt):
                hit = False
                stop = i+max_size if len(txt) - (i + max_size) >= 0 else len(txt)
                while hit == False and stop > i:
                    if txt[i:stop] in self.itos:
                        new_txt.append(self.stoi[txt[i:stop]])
                        hit = True
                        i = stop
                    else:
                        stop -= 1
            new_text.append(new_txt)
                                
        text = new_text
       # for token_id, token in enumerate(self.itos):
            # find occurences of the token in the text and replace them with token_id
          #  text = # TODO       
        return text
    
    def decode_token(self, tok):
        """
        tok: int or tuple
        """
        result = self.itos[tok] if isinstance(tok, int) else [self.itos[i] for i in token]
        return result
            
    def decode(self, text):
        """
        convert token ids into text
        """
        return ''.join(map(self.decode_token, text))
        
        
vocab_size = 100
bpe = BPE(vocab_size)
tokenized_text = bpe.fit_transform(text)

In [0]:
assert bpe.decode(tokenized_text[0]) == text[0]

In [98]:
bpe.decode(tokenized_text[0]) 

'the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.'

In [99]:
text[0]

'the project gutenberg ebook of war and peace by leo tolstoy this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.'

In [100]:
print(tokenized_text[0])

[66, 55, 26, 15, 7, 24, 28, 62, 12, 34, 45, 70, 38, 65, 74, 24, 38, 15, 15, 48, 30, 84, 27, 83, 30, 72, 55, 24, 49, 28, 58, 38, 67, 1, 24, 69, 45, 15, 1, 90, 15, 67, 59, 29, 61, 24, 38, 15, 15, 48, 30, 29, 61, 11, 76, 30, 66, 34, 43, 58, 84, 64, 9, 68, 58, 64, 9, 27, 56, 65, 58, 85, 19, 69, 28, 15, 90, 30, 72, 27, 29, 59, 30, 88, 57, 15, 90, 30, 19, 69, 80, 90, 26, 29, 28, 45, 29, 68, 61, 27, 79, 45, 43, 15, 24, 21, 65, 0]


In [0]:
start_token = vocab_size
end_token = vocab_size + 1
        
    
class LM:
    
    def __init__(self, vocab_size, delta=1):
        self.delta = delta
        self.vocab_size = vocab_size + 2
        self.proba = Counter() # TODO create array for storing 3-gram counters
        
    def infer(self, a, b, tau=1):
        """
        return vector of probabilities of size self.vocab for 3-grams which start with (a,b) tokens
        a: first token id
        b: second token id
        tau: temperature
        """
        result = []
        for item in range(self.vocab_size):
            result.append(self.get_proba(a, b, item, tau))
        return result
        
    def get_proba(self, a, b, c, tau=1):
        """
        get probability of 3-gram (a,b,c)
        a: first token id
        b: second token id
        c: third token id
        tau: temperature
        """
        # TODO approbximate probability by counters
        
        def smooth(p, delta=self.delta, tau=tau):
          return (p + delta) ** (1 / tau)
        
        P_abc = smooth(self.proba[(a, b, c)])
        P_ab = sum([smooth(self.proba[(a,b,x)]) 
                    for x in range(self.vocab_size)])
        
        return math.log(P_abc / P_ab)
    
    def fit(self, text):
        """
        train language model on text
        text: list of lists
        """
        self.text = text
        for txt in text:
            for i in range(len(txt) - 2):
                self.proba[txt[i], txt[i+1], txt[i+2]] += 1
                                       
        # self.proba = # TODO count 3-grams in the text
    
        return self
    
lm = LM(vocab_size, 1).fit(tokenized_text)

In [0]:
def beam_search(input_seq, lm, max_len=10, k=5, tau=1):
    """
    generate sequence from language model *lm* conditioned on input_seq
    input_seq: sequence of token ids for conditioning
    lm: language model
    max_len: max generated sequence length
    k: size of beam
    tau: temperature
    """
    # TODO store in beam tuples of current sequences and their log probabilities
    
   
    if len(input_seq) >= 3:
      
      prob = lm.get_proba(input_seq[0], input_seq[1] ,input_seq[2], tau)
      
      i = 1
      while i + 2 < len(input_seq):
        prob += lm.get_proba(input_seq[i], input_seq[i + 1] ,input_seq[i + 2], tau)
        i += 1
      
      proba = enumerate(lm.infer(input_seq[-2], input_seq[-1], tau))
      proba = sorted(proba, key=lambda x: x[1], reverse=True)[:k]
      
      beam = {}
      for i in range(k):
        keys = list(input_seq)
        keys.append(proba[i][0])
        keys = tuple(list(keys))
        beam[keys] = proba[i][1] + prob
        
    elif len(input_seq) == 2:
      beam = [item for item in lm.proba.keys() if item[0] == input_seq[-2] \
                and item[1] == input_seq[-1]]
      beam = {item: lm.get_proba(item[0], item[1], item[2], tau) for item in beam}   
    else:
        beam = [item for item in lm.proba.keys() if item[0] == input_seq[0]]
        beam = {item: lm.get_proba(item[0], item[1], item[2], tau) for item in beam}   
            
    beam = sorted(beam.items(), key= lambda x: x[1], reverse=True)[:k]
    beam = {item[0] : item[1] for item in beam}
    
    for i in range(max_len):
      if sum ([1 for item in beam.keys() if item[-1] != 0]) == 0:
        break
      candidates = []
      candidates_proba = []
      for snt, snt_proba in beam.items():          
        key = list(snt)
        if 0 in snt:
          candidates.append(tuple(key))
          candidates_proba.append(snt_proba)
            # TODO process terminal token 
        else:   
          proba = {i: j for i, j in enumerate(lm.infer(snt[-2], snt[-1]))} # probability vector of the next token
          best_k = sorted(proba.items(), key=lambda x: x[1], reverse=True)
          for item in best_k:
            candidates_proba.append(item[1] + snt_proba)
            key = list(snt)
            key.append(item[0])
            candidates.append(tuple(key))
      beam = {candidates[a]: candidates_proba[a] for a in range(len(candidates))}
      beam = sorted(beam.items(), key=lambda x: x[1], reverse=True)[:k]
      beam = {i[0]: i[1] for i in beam}
    #  print(i, beam)
                   
    return beam
   

In [103]:
input1 = 'horse '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

for res in result.items():
  print(bpe.decode(tuple([i for i in res[0] if i != '.'])), math.exp(res[1]))
    

horse with him. 0.00020842031405734197
horse withought. 9.801724730010705e-05
horse with himself. 1.4255371501910611e-05
horse with with him  5.413034675990682e-06
horse with himselves  4.7965209699688785e-06
horse with himself wi 2.960555734184556e-06
horse with himselvem 2.7151903715602943e-06
horse with himself bu 1.6475059458537927e-06
horse with with hims 1.4977529550640893e-06
horse with himselve. 1.4001678571112323e-06


In [109]:
input1 = 'her'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

for res in result.items():
  print(bpe.decode(tuple([i for i in res[0] if i != '.'])), math.exp(res[1]))

here with him. 9.796432005746645e-05
here withought. 4.607129122268615e-05
here with himself. 6.70048782273228e-06
here with himselve 6.496754959727887e-06
here with him with 5.861138738160363e-06
here with himself and  5.8163297627770094e-06
here with himself w 5.306125397621135e-06
here with himself to  4.217689418621928e-06
here with himself b 3.0612261909352725e-06
here with himself in  2.8911580692166416e-06


In [110]:
input1 = 'what'
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=1)
# TODO print decoded generated strings and their probabilities

for res in result.items():
  print(bpe.decode(tuple([i for i in res[0] if i != '.'])), math.exp(res[1]))

what. 0.0021381242623291935
whated. 0.0007596152567260526
whates. 0.0003372007833410722
whats with him. 8.988461381837363e-07
whatess with him. 7.531022157368436e-08
whatess with himsel 7.44891126994022e-08
whats with himself. 6.147858322169853e-08
whats with himselve 5.9609285328086e-08
whats with himself and  5.336622091194494e-08
whats with himself w 4.868497346352874e-08


In [112]:
input1 = 'gun '
input1 = bpe.transform([input1])[0]
result = beam_search(input1, lm, max_len=10, k=10, tau=0.1)
# TODO print decoded generated strings and their probabilities

for res in result.items():
  print(bpe.decode(tuple([i for i in res[0] if i != '.'])), math.exp(res[1]))

gun with him. 6.955427047028502e-11
gun withought. 3.2710430172313876e-11
gun with himself. 4.757319216136916e-12
gun with with him  1.8064442500326712e-12
gun with himselves  1.6007005764794181e-12
gun with himself wi 9.880001150999742e-13
gun with himselvem 9.06116499900577e-13
gun with himself bu 5.49807607178781e-13
gun with with hims 4.998318643044331e-13
gun with himselve. 4.672656515166748e-13


In [0]:
def perplexity(snt, lm):
    """
    snt: sequence of token ids
    lm: language model
    """
    i = 1
    P = lm.get_proba(snt[0], snt[1], snt[2])
    while i < len(snt) - 2:
      P += lm.get_proba(snt[i], snt[i + 1], snt[i + 2])
      i += 1
    P = math.exp(P)
    t = -1 *(1 / len(snt))

    return P ** t

In [117]:
perplexity(tokenized_text[0], lm)


12.53881439694464